In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
import re
from gensim.models import Word2Vec

Input Analysis

What are the length distributions of documents and sentences? Any outliers?

In [ ]:
Image("../data/eda/outlier.png")

Boxplot Visual would give a better view of the length of documents.

In [ ]:
Image("../data/eda/boxplot.png")

After removing the outliers..

In [ ]:
Image("../data/eda/cleaned_set.png")

What is the distribution of all tokens? How many "rare" tokens (e.g., < 5 times)?

In [ ]:
Image("../data/eda/tokens_distribution.png")

In [ ]:
f = open("../data/eda/description.txt", "r")
print(f.read())

Output Analysis

Run AutoPhrase, and then plot the quality score distribution of single-word and multi-word phrases separately.

In [ ]:
Image("../data/outputs/multi_value_distribution.png")

In [ ]:
Image("../data/outputs/single_value_distribution.png")

In this case, we use reference sample to fix the sample to do the similar task. But if you want to your own sample, it is in the output folder by naming sample.txt. You should manually value the high-quality phrases and add the column in the dataset. Then you can change the configuration sample and word and rerun the example part.


In our fixed sample, we mannually label all high quality phrase with 1 and 0 otherwise and called the file sample_example.txt. The model word2vec code on the phrasal segmentation results to obtain phrase embedding is in phrase_embedding.model. 
The precision-recall curve shown as below:

In [ ]:
Image("../data/outputs/example/precision-recall.png")

Pick 3 high-quality phrases from your previous annotations in step 1, run a similarity search among all multi-word phrases whose scores are greater than 0.5, and report the top-5 results. 

3 high-quality phrases in our sample are: web_publishing, competitive_factor, and uncertain_parameter


The most five similar phrases with values are:

In [ ]:
pd.read_csv("../data/outputs/example/most_similar.txt",header = None,sep=' ', names 
            = ['similar_words','value'])

Furthermore, the tokens to train the Word2Vec model are stored in the word.txt file.